### **Inisialisasi**

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Inisialisasi SparkSession
spark = SparkSession.builder.appName('HandsOnPertemuan3').getOrCreate()

# Data dan skema dasar
data = [
    ('James', 'Sales', 3000),
    ('Michael', 'Sales', 4600),
    ('Robert', 'Sales', 4100),
    ('Maria', 'Finance', 3000),
    ('Ravi', 'Finance', 4000)
]
columns = ['EmployeeName', 'Department', 'Salary']

### **Tugas 1: Buat DataFrame sederhana di Spark dan eksplorasi beberapa fungsi dasar yang tersedia.**

In [ ]:
# Tugas 1: Buat DataFrame sederhana dan eksplorasi fungsi dasar
df = spark.createDataFrame(data, schema=columns)
print("DataFrame sederhana dan fungsi dasar")
df.show()
df.printSchema()
df.describe().show()

DataFrame sederhana dan fungsi dasar
+------------+----------+------+
|EmployeeName|Department|Salary|
+------------+----------+------+
|       James|     Sales|  3000|
|     Michael|     Sales|  4600|
|      Robert|     Sales|  4100|
|       Maria|   Finance|  3000|
|        Ravi|   Finance|  4000|
+------------+----------+------+

root
 |-- EmployeeName: string (nullable = true)
 |-- Department: string (nullable = true)
 |-- Salary: long (nullable = true)

+-------+------------+----------+-----------------+
|summary|EmployeeName|Department|           Salary|
+-------+------------+----------+-----------------+
|  count|           5|         5|                5|
|   mean|        NULL|      NULL|           3740.0|
| stddev|        NULL|      NULL|712.7411872482185|
|    min|       James|   Finance|             3000|
|    max|      Robert|     Sales|             4600|
+-------+------------+----------+-----------------+



### **Tugas 2: Gunakan operasi filter, select, groupBy untuk mengekstrak informasi dari data, serta lakukan agregasi data untuk mendapatkan insight tentang dataset menggunakan perintah seperti mean, max, sum.**

In [ ]:
# Tugas 2: Operasi filter, select, groupBy dan agregasi (mean, max, sum)
df.select('EmployeeName', 'Salary').show() # Select kolom tertentu
df.filter(df['Salary'] > 3000).show() # Filter gaji > 3000
df.groupBy('Department').agg(
    F.mean('Salary').alias('MeanSalary'),
    F.max('Salary').alias('MaxSalary'),
    F.sum('Salary').alias('TotalSalary')
).show()

+------------+------+
|EmployeeName|Salary|
+------------+------+
|       James|  3000|
|     Michael|  4600|
|      Robert|  4100|
|       Maria|  3000|
|        Ravi|  4000|
+------------+------+

+------------+----------+------+
|EmployeeName|Department|Salary|
+------------+----------+------+
|     Michael|     Sales|  4600|
|      Robert|     Sales|  4100|
|        Ravi|   Finance|  4000|
+------------+----------+------+

+----------+----------+---------+-----------+
|Department|MeanSalary|MaxSalary|TotalSalary|
+----------+----------+---------+-----------+
|     Sales|    3900.0|     4600|      11700|
|   Finance|    3500.0|     4000|       7000|
+----------+----------+---------+-----------+



### **Tugas 3: Eksplorasi bagaimana mengolah tipe data kompleks dalam Spark DataFrames.**

In [ ]:
# Tugas 3: Eksplorasi tipe data kompleks, menambah kolom kolom baru dengan perhitungan
df = df.withColumn('SalaryBonus', df['Salary'] * 0.1) # Bonus 10% salary
df = df.withColumn('TotalCompensation', df['Salary'] + df['SalaryBonus']) # Total kompensasi
df.show()

+------------+----------+------+-----------+-----------------+
|EmployeeName|Department|Salary|SalaryBonus|TotalCompensation|
+------------+----------+------+-----------+-----------------+
|       James|     Sales|  3000|      300.0|           3300.0|
|     Michael|     Sales|  4600|      460.0|           5060.0|
|      Robert|     Sales|  4100|      410.0|           4510.0|
|       Maria|   Finance|  3000|      300.0|           3300.0|
|        Ravi|   Finance|  4000|      400.0|           4400.0|
+------------+----------+------+-----------+-----------------+



### **Tugas 4: Implementasikan window function untuk menghitung running totals atau rangkings.**

In [ ]:
# Tugas 4: Implementasi window function untuk rangking berdasarkan Salary dalam Department
print("rank berdasarkan Salary per Department")
windowSpec = Window.partitionBy('Department').orderBy(df['Salary'].desc())
df.withColumn('Rank', F.rank().over(windowSpec)).show()

rank berdasarkan Salary per Department
+------------+----------+------+-----------+-----------------+----+
|EmployeeName|Department|Salary|SalaryBonus|TotalCompensation|Rank|
+------------+----------+------+-----------+-----------------+----+
|        Ravi|   Finance|  4000|      400.0|           4400.0|   1|
|       Maria|   Finance|  3000|      300.0|           3300.0|   2|
|     Michael|     Sales|  4600|      460.0|           5060.0|   1|
|      Robert|     Sales|  4100|      410.0|           4510.0|   2|
|       James|     Sales|  3000|      300.0|           3300.0|   3|
+------------+----------+------+-----------+-----------------+----+



### **Tugas 5: Eksplorasi dengan Dataset**

In [8]:
# persiapan

from google.colab import files
files.upload()  # Upload kaggle.json

!mkdir -p ~/.kaggle     # instalasi dan setup
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!pip install kaggle

!kaggle datasets download aprabowo/indonesia-tourism-destination
!unzip indonesia-tourism-destination.zip -d indonesia_tourism

Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/aprabowo/indonesia-tourism-destination
License(s): copyright-authors
  0% 0.00/158k [00:00<?, ?B/s]
100% 158k/158k [00:00<00:00, 341MB/s]
Archive:  indonesia-tourism-destination.zip
  inflating: indonesia_tourism/package_tourism.csv  
  inflating: indonesia_tourism/tourism_rating.csv  
  inflating: indonesia_tourism/tourism_with_id.csv  
  inflating: indonesia_tourism/user.csv  


In [11]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# Inisialisasi SparkSession
spark = SparkSession.builder.appName('IndonesiaTourism').getOrCreate()

# Load CSV ke DataFrame dengan header dan inferSchema aktif supaya kolom sesuai tipe data
package_df = spark.read.option('header', True).option('inferSchema', True).csv('indonesia_tourism/package_tourism.csv')
rating_df = spark.read.option('header', True).option('inferSchema', True).csv('indonesia_tourism/tourism_rating.csv')
tourism_df = spark.read.option('header', True).option('inferSchema', True).csv('indonesia_tourism/tourism_with_id.csv')
user_df = spark.read.option('header', True).option('inferSchema', True).csv('indonesia_tourism/user.csv')

In [13]:
# Tampilkan skema singkat sebagai validasi
print("Schema package_tourism:")
package_df.printSchema()

print("Schema tourism_rating:")
rating_df.printSchema()

Schema package_tourism:
root
 |-- Package: integer (nullable = true)
 |-- City: string (nullable = true)
 |-- Place_Tourism1: string (nullable = true)
 |-- Place_Tourism2: string (nullable = true)
 |-- Place_Tourism3: string (nullable = true)
 |-- Place_Tourism4: string (nullable = true)
 |-- Place_Tourism5: string (nullable = true)

Schema tourism_rating:
root
 |-- User_Id: integer (nullable = true)
 |-- Place_Id: integer (nullable = true)
 |-- Place_Ratings: integer (nullable = true)



In [15]:
# eksplorasi dan tampilkan isi data

# eksplorasi untuk package_tourism
print("Data sample package_tourism:")
package_df.show(5)
print("Statistik deskriptif numeric:")
package_df.describe().show()

# Eksplorasi tourism_rating
print("Data sample tourism_rating:")
rating_df.show(5)
print("Statistik deskriptif numeric:")
rating_df.describe().show()

# Eksplorasi tourism_with_id
print("Data sample tourism_with_id:")
tourism_df.show(5)
print("Kategori unik destinasi wisata:")
tourism_df.select('Category').distinct().show()

# Eksplorasi user data
print("Data sample user:")
user_df.show(5)
print("Jumlah total user unik:")
print(user_df.select('User_id').distinct().count())

Data sample package_tourism:
+-------+-------+--------------------+------------------+--------------------+--------------------+--------------+
|Package|   City|      Place_Tourism1|    Place_Tourism2|      Place_Tourism3|      Place_Tourism4|Place_Tourism5|
+-------+-------+--------------------+------------------+--------------------+--------------------+--------------+
|      1|Jakarta|   Pasar Tanah Abang|      Taman Ayodya|      Museum Tekstil|                NULL|          NULL|
|      2|Jakarta|   Pasar Tanah Abang|Pasar Taman Puring|Pasar Petak Sembilan|                NULL|          NULL|
|      3|Jakarta|Perpustakaan Nasi...|             Monas|     Masjid Istiqlal|                NULL|          NULL|
|      4|Jakarta|        Pulau Tidung|    Pulau Bidadari|          Pulau Pari|       Pulau Pramuka| Pulau Pelangi|
|      5|Jakarta|Museum Satria Man...|     Museum Wayang|Museum Bahari Jak...|Museum Macan (Mod...|          NULL|
+-------+-------+--------------------+-------------

In [17]:
print(tourism_df.columns)

['Place_Id', 'Place_Name', 'Description', 'Category', 'City', 'Price', 'Rating', 'Time_Minutes', 'Coordinate', 'Lat', 'Long', '_c11', '_c12']


In [36]:
# manipulasi lanjutan

# Tampilkan 5 baris awal sebagai referensi
print("isi data 5 baris awal:")
tourism_df.show(5)

# Isi nilai null pada Rating dengan 0
tourism_df = tourism_df.fillna({'Rating': 0})

# Filter destinasi dengan harga tiket di bawah 50000 dan rating di atas 4
print("Destinasi dengan harga tiket di bawah 50000 dan rating di atas 4:")
filtered_df = tourism_df.filter((F.col('Price') < 50000) & (F.col('Rating') > 4))
filtered_df.show()

# Hitung rating tertinggi dan jumlah destinasi wisata per kota
print("Rating tertinggi dan jumlah destinasi wisata per kota:")
rating_city_stats = tourism_df.groupBy('City').agg(
    F.max('Rating').alias('MaxRating'),
    F.count('Place_Id').alias('CountDestinations')
)
rating_city_stats.show()

# Urutkan destinasi berdasarkan rating tertinggi dan harga termurah (descending rating, ascending price)
print("Destinasi berdasarkan rating tertinggi dan harga termurah:")
sorted_df = tourism_df.orderBy(F.col('Rating').desc(), F.col('Price').asc())
sorted_df.select('Place_Name', 'Rating', 'Price').show(17)

isi data 5 baris awal:
+--------+--------------------+--------------------+-------------+-------+------+------+------------+--------------------+----------+-----------+----+----+
|Place_Id|          Place_Name|         Description|     Category|   City| Price|Rating|Time_Minutes|          Coordinate|       Lat|       Long|_c11|_c12|
+--------+--------------------+--------------------+-------------+-------+------+------+------------+--------------------+----------+-----------+----+----+
|       1|    Monumen Nasional|Monumen Nasional ...|       Budaya|Jakarta| 20000|   4.6|          15|{'lat': -6.175392...|-6.1753924|106.8271528|NULL|   1|
|       2|            Kota Tua|Kota tua di Jakar...|       Budaya|Jakarta|     0|   4.6|          90|{'lat': -6.137644...|-6.1376448|106.8171245|NULL|   2|
|       3|       Dunia Fantasi|Dunia Fantasi ata...|Taman Hiburan|Jakarta|270000|   4.6|         360|{'lat': -6.125312...|-6.1253124|106.8335377|NULL|   3|
|       4|Taman Mini Indone...|Taman Mini